In [1]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import AdamW
from sklearn.metrics import accuracy_score
from datasets import load_dataset
from torch.utils.data import DataLoader

In [ ]:
# Load pre-trained model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Set device to the MPS device
    print("Running on the MPS device")
else:
    device = torch.device("cpu")  # Set device to CPU
    print("Running on the CPU")

# Move your model to the device
model.to(device)

In [ ]:
# Load the dataset
raw_datasets = load_dataset("glue", "sst2")

# Define a function to tokenize the data
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Move the datasets to the device
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'], device=device)

In [ ]:
# Prepare the data loaders
train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=8, shuffle=True)

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Define the training loop
for epoch in range(3):  # Number of epochs
    for batch in train_dataloader:
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()